In [1]:
%matplotlib inline
import importlib
import numpy as np
import pandas as pd
import json
import utils

In [2]:
importlib.reload(utils)

<module 'utils' from '/home/fei/Documents/projects/lyrics/encoder-decoder/utils.py'>

In [23]:
dat = pd.read_csv('../songdata.csv', usecols=['text'], nrows=100).drop_duplicates()

In [24]:
dat.head()

,text
0,"Look at her face, it's a wonderful face \nAnd..."
1,"Take it easy with me, please \nTouch me gentl..."
2,I'll never know why I had to go \nWhy I had t...
3,Making somebody happy is a question of give an...
4,Making somebody happy is a question of give an...


In [25]:
dat.loc[:, 'text'] += '(end)'

In [26]:
unique_chars = dat.loc[:, 'text'].apply(lambda x: list(set(x)))

In [27]:
unique_chars.head()

0    [ , y, H, h, S, (, ', ,, \n, d, p, ?, s, i, a,...
1    [ , y, G, h, P, (, Y, ', p, \n, ,, d, s, i, a,...
2    [ , y, G, H, h, S, Y, (, ', p, \n, d, ,, s, i,...
3    [ , y, h, Y, S, P, ', (, E, p, \n, d, ,, s, i,...
4    [ , y, h, Y, S, P, ', (, E, p, \n, d, ,, s, i,...
Name: text, dtype: object

In [28]:
unique_chars = unique_chars.sum()

In [29]:
unique_chars = sorted(np.unique(unique_chars))

In [30]:
def scanning(text, length):
    encoder_input = []
    decoder_input = []
    text_length = len(text)
    for i in range(0, text_length-2*length+1):
        encoder_input.append(list(text[i:i+length]))
        decoder_input.append(list(text[i+length:i+2*length]))
    return encoder_input, decoder_input

In [33]:
dat_scanned = dat.text.apply(lambda text: pd.Series(scanning(text, length=100), index=['for_encoder', 'for_decoder']))

In [34]:
dat_scanned.head()

,for_encoder,for_decoder
0,"[[L, o, o, k, , a, t, , h, e, r, , f, a, c,...","[[t, , s, h, e, , s, m, i, l, e, s, , w, h,..."
1,"[[T, a, k, e, , i, t, , e, a, s, y, , w, i,...","[[t, , s, l, o, w, , , \n, A, n, d, a, n, t..."
2,"[[I, ', l, l, , n, e, v, e, r, , k, n, o, w,...","[[a, c, k, i, n, g, , a, l, l, , m, y, , s,..."
3,"[[M, a, k, i, n, g, , s, o, m, e, b, o, d, y,...","[[ , y, o, u, r, s, e, l, f, , a, , b, r, e,..."
4,"[[M, a, k, i, n, g, , s, o, m, e, b, o, d, y,...","[[ , y, o, u, r, s, e, l, f, , a, , b, r, e,..."


In [37]:
for_encoder = dat_scanned.for_encoder.sum()

for_decoder = dat_scanned.for_decoder.sum()

In [55]:
for_encoder_se = pd.Series(for_encoder)
for_decoder_se = pd.Series(for_decoder)

In [57]:
for_encoder_se.head()

0    [L, o, o, k,  , a, t,  , h, e, r,  , f, a, c, ...
1    [o, o, k,  , a, t,  , h, e, r,  , f, a, c, e, ...
2    [o, k,  , a, t,  , h, e, r,  , f, a, c, e, ,, ...
3    [k,  , a, t,  , h, e, r,  , f, a, c, e, ,,  , ...
4    [ , a, t,  , h, e, r,  , f, a, c, e, ,,  , i, ...
dtype: object

In [58]:
for_decoder_se.head()

0    [t,  , s, h, e,  , s, m, i, l, e, s,  , w, h, ...
1    [ , s, h, e,  , s, m, i, l, e, s,  , w, h, e, ...
2    [s, h, e,  , s, m, i, l, e, s,  , w, h, e, n, ...
3    [h, e,  , s, m, i, l, e, s,  , w, h, e, n,  , ...
4    [e,  , s, m, i, l, e, s,  , w, h, e, n,  , s, ...
dtype: object

In [44]:
utils.save_unique_tokens(unique_chars)

In [45]:
word2ind, ind2word = utils.get_index_word_map(unique_chars)

In [46]:
utils.save_index_word_map(word2ind, ind2word)

In [62]:
encoder_input_tokenised = utils.tokenise_cleaned_data(for_encoder_se, word2ind)

In [63]:
encoder_input_tokenised.head()

0    [25, 54, 54, 50, 1, 40, 59, 1, 47, 44, 57, 1, ...
1    [54, 54, 50, 1, 40, 59, 1, 47, 44, 57, 1, 45, ...
2    [54, 50, 1, 40, 59, 1, 47, 44, 57, 1, 45, 40, ...
3    [50, 1, 40, 59, 1, 47, 44, 57, 1, 45, 40, 42, ...
4    [1, 40, 59, 1, 47, 44, 57, 1, 45, 40, 42, 44, ...
dtype: object

In [64]:
decoder_input_tokenised = utils.tokenise_cleaned_data(for_decoder_se, word2ind)

In [65]:
decoder_input_tokenised.head()

0    [59, 1, 58, 47, 44, 1, 58, 52, 48, 51, 44, 58,...
1    [1, 58, 47, 44, 1, 58, 52, 48, 51, 44, 58, 1, ...
2    [58, 47, 44, 1, 58, 52, 48, 51, 44, 58, 1, 62,...
3    [47, 44, 1, 58, 52, 48, 51, 44, 58, 1, 62, 47,...
4    [44, 1, 58, 52, 48, 51, 44, 58, 1, 62, 47, 44,...
dtype: object

In [66]:
decoder_target_tokenised = decoder_input_tokenised.apply(lambda x: x[1:])

In [67]:
decoder_target_tokenised.head()

0    [1, 58, 47, 44, 1, 58, 52, 48, 51, 44, 58, 1, ...
1    [58, 47, 44, 1, 58, 52, 48, 51, 44, 58, 1, 62,...
2    [47, 44, 1, 58, 52, 48, 51, 44, 58, 1, 62, 47,...
3    [44, 1, 58, 52, 48, 51, 44, 58, 1, 62, 47, 44,...
4    [1, 58, 52, 48, 51, 44, 58, 1, 62, 47, 44, 53,...
dtype: object

In [68]:
len(decoder_target_tokenised.loc[0])

99

In [69]:
decoder_input_tokenised = decoder_input_tokenised.apply(lambda x: x[:-1])

In [70]:
decoder_input_tokenised.head()

0    [59, 1, 58, 47, 44, 1, 58, 52, 48, 51, 44, 58,...
1    [1, 58, 47, 44, 1, 58, 52, 48, 51, 44, 58, 1, ...
2    [58, 47, 44, 1, 58, 52, 48, 51, 44, 58, 1, 62,...
3    [47, 44, 1, 58, 52, 48, 51, 44, 58, 1, 62, 47,...
4    [44, 1, 58, 52, 48, 51, 44, 58, 1, 62, 47, 44,...
dtype: object

In [71]:
len(decoder_input_tokenised.iloc[0])

99

In [72]:
encoder_input_tokenised = pd.DataFrame(encoder_input_tokenised.values.tolist())
decoder_input_tokenised = pd.DataFrame(decoder_input_tokenised.values.tolist())
decoder_target_tokenised = pd.DataFrame(decoder_target_tokenised.values.tolist())

In [73]:
encoder_input_tokenised.shape

(114504, 100)

In [74]:
decoder_input_tokenised.shape

(114504, 99)

In [75]:
decoder_target_tokenised.shape

(114504, 99)

In [76]:
encoder_input_tokenised.to_csv('encoder_input.csv', index=False)
decoder_input_tokenised.to_csv('decoder_input.csv', index=False)
decoder_target_tokenised.to_csv('decoder_target.csv', index=False)